In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)
from datetime import datetime
import tkinter as tk
from tkinter import filedialog
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
def raw_data_into_list(line_data):
    if line_data[:15] != '               ':
        line_data = line_data.strip()
    else:
        line_data = line_data.rstrip()
    character_number = [15, 8, 5, 4, 20, 6, 1, 5, 20, 4, 2, 64]
    variable = []
    x, count, position = 0, 0, 0
    for letter in line_data:
        position += 1
        if '\u4e00' <= letter <= '\u9fa5':  # 中文2字元
            count += 2
        else:  # 非中文1字元
            count += 1
        if count >= character_number[x]:
            character = line_data[:position]  # 取出的字元資料
            if character.strip() == '':  # 將無資料欄位填入na，再後續處理
                variable.append("na")
            else:
                variable.append(character.strip())
            line_data = line_data.replace(character, '', 1)
            count, position = 0, 0
            x += 1  # 字元數陣列定位點
    return variable

In [3]:
def read_raw_data(file_path):
    test = pd.DataFrame([], columns=["data"])
    with open(file_path, "r") as f:
        data = f.readlines()
        data_list = [raw_data_into_list(i) for i in data[1:]]
        test["data"] = data_list
        return test
        

In [9]:
gender = {"1": "男", "2": "女", "X": "法人"}
def combine_data(df):
    result = pd.DataFrame(df["data"].tolist(), columns=["品牌", "發照日", "戶籍郵遞區號", "車種", "車型名稱", "出廠年月", "性別", "車重", "輪胎尺寸", "車主出生年", "原始監理單位代碼", "引擎與車身號碼加密"])
    id_num = "AAA261A6E8EE9EC7D61EFCB00153B47AFB7FB06FC07B7E924B92F9C40AEEC0D3"
    result = result[result["引擎與車身號碼加密"] != id_num]
    result = result.drop_duplicates(subset="引擎與車身號碼加密", keep="last")
    result["發照日"] = pd.to_datetime(result["發照日"].map(lambda x: str(int(x[0:3]) + 1911) + x[3:]), yearfirst=True)
    result["出廠年月"] = pd.to_datetime(result["出廠年月"].map(lambda x: x[:4] + "-" + x[4:] + "-" + "01"), yearfirst=True, errors="coerce")
    result["出廠年月"] = result.apply(lambda x: x["發照日"] if x["出廠年月"] > x["發照日"] else x["出廠年月"], axis=1)
    result["車主出生年"] = result["車主出生年"].map(lambda x: str(int(x) + 1911) if x != "000" and len(x) == 3 else x)
    result["性別"] = result["性別"].map(gender)
    return result

In [ ]:
car_type = pd.read_excel(r"C:\Users\kc.hsu\Python workspace\sales_related\New Car Registration Data\cross_tables\CAR_TYPE_INDEX.xlsx")

In [ ]:
car_type1 = dict(zip(car_type["車種代碼"].astype(str), car_type["車輛類型1"]))
car_type2 = dict(zip(car_type["車種代碼"].astype(str), car_type["車輛類型2"]))
car_type3 = dict(zip(car_type["車種代碼"].astype(str), car_type["使用類型"]))

In [ ]:
post_index = pd.read_excel(r"C:\Users\kc.hsu\Python workspace\sales_related\New Car Registration Data\cross_tables\ZIP_INDEX.xlsx")

In [ ]:
city_dict = dict(zip(post_index["CODE"].astype(str), post_index["CITY"]))
district_dict = dict(zip(post_index["CODE"].astype(str), post_index["DISTRICT"]))
area_dict = dict(zip(post_index["CODE"].astype(str), post_index["AREA"]))

In [ ]:
gov = pd.read_excel(r"C:\Users\kc.hsu\Python workspace\sales_related\New Car Registration Data\cross_tables\MOV_INDEX.xlsx")

In [ ]:
gov_dict = dict(zip(gov["監理站代碼"].astype(str), gov["監理站"]))

In [ ]:
brand = pd.read_excel("./cross_tables/BRAND_INDEX.xlsx")
brand1_dict = dict(zip(brand["BRAND_RAW"], brand["BRAND_1"]))
brand2_dict = dict(zip(brand["BRAND_RAW"], brand["BRAND_2"]))
source_dict = dict(zip(brand["BRAND_RAW"], brand["SOURCE"]))

In [ ]:
models = pd.read_excel("./cross_tables/20220822_MODEL_INDEX.xlsx")
model_dict = dict(zip(models["MODEL_RAW"], models["MODEL"]))

In [ ]:
tire_size = pd.read_excel("./cross_tables/20220822_TIRE_SIZE_INDEX_NEW.xlsx")
tire_size_dict = dict(zip(tire_size["輪胎尺寸"], tire_size["Extract tire size 2"]))

## **原始檔轉換區**

### 一次多檔案合併

In [ ]:
path = [r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20200901_20200930",
        r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20201001_20201031", r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20201101_20201130",
        r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20201201_20201231", r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20210101_20210131",
        r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20210201_20210228", r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20210301_20210331",
        r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20210401_20210430", r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20210501_20210531",
        r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20210601_20210630", r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20210701_20210731",
        r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20210801_20210831", r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20210901_20210930",
        r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20211001_20211031", r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20211101_20211130",
        r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20211201_20211231", r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20220101_20220128",
        r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20220201_20220228", r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20220301_20220331",
        r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20220401_20220430", r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20220501_20220531",
        r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20220601_20220630", r"D:\kc.hsu\OneDrive - Bridgestone\程式碼\raw_data\bs_20220701_20220731"]

In [ ]:
data_list = []
for p, n in zip(path, range(len(path))):
    globals()["data_" + str(n)] = read_raw_data(p)
    globals()["data_" + str(n)] = combine_data(globals()["data_" + str(n)])
    data_list.append(globals()["data_" + str(n)])

In [ ]:
result = pd.concat(data_list).reset_index(drop=True)

### 單一檔案合併

In [4]:
root = tk.Tk()
root.withdraw()
path = filedialog.askopenfilename()

In [21]:
df = read_raw_data(path)

In [22]:
result = combine_data(df)

In [ ]:
result

In [ ]:
result['引擎與車身號碼加密'].map(lambda x: len(x))

### 後續處理

In [ ]:
result["車型名稱"] = result["車型名稱"].map(lambda x: x.strip())

In [ ]:
result["車輛類型1"] = result["車種"].map(car_type1)
result["車輛類型2"] = result["車種"].map(car_type2)
result["使用類型"] = result["車種"].map(car_type3)

In [ ]:
result["城市"] = result["戶籍郵遞區號"].map(city_dict)
result["區"] = result["戶籍郵遞區號"].map(district_dict)
result["區域"] = result["戶籍郵遞區號"].map(area_dict)

In [ ]:
result["監理站"] = result["原始監理單位代碼"].map(gov_dict)

In [ ]:
result["品牌1"] = result["品牌"].map(brand1_dict)
result["品牌2"] = result["品牌"].map(brand2_dict)
result["來源"] = result["品牌"].map(source_dict)

In [ ]:
result["車型"] = result["車型名稱"].map(model_dict)

In [ ]:
result["Tire size"] = result["輪胎尺寸"].map(tire_size_dict)

In [ ]:
result.to_excel(r"C:\Users\kc.hsu\Desktop\20221020監理站數字.xlsx", index=False)

In [ ]:
result = result[["品牌1", "品牌2", "來源", "車型", "Tire size", "發照日", "出廠年月", "性別", "車重", "車主出生年", "車輛類型1", "車輛類型2", "使用類型", "城市", "區", "區域", "監理站"]]

In [ ]:
result["發照日"] = result["發照日"].dt.date
result["出廠年月"] = result["出廠年月"].dt.date

In [ ]:
result.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\監理站新車領牌數據\20220915_2022年9月數據.xlsx", index=False)

In [ ]:
data = pd.read_csv(r"D:\kc.hsu\OneDrive - Bridgestone\數據\監理站新車領牌數據\20220915_監理站領牌數據全.csv")

In [ ]:
final = pd.concat([data, result])

In [ ]:
big_cat_keys = final["車輛類型2"].unique().tolist()
big_cat_values = ['PSV', 'LCV', 'PSV', 'Truck/Bus', 'Truck/Bus', 'Truck/Bus', 'PSV', 'Truck/Bus', 'Truck/Bus',
       'Truck/Bus', 'LCV', 'Truck/Bus']
big_cat_dict = dict(zip(big_cat_keys, big_cat_values))

In [ ]:
final["車輛類型3"] = final["車輛類型2"].map(big_cat_dict)

In [ ]:
export_path = "D:/kc.hsu/OneDrive - Bridgestone/數據/監理站新車領牌數據/"
# file_name = path.split("\\")[-1].split("_")[-1][:-2] + "_監理資料.xlsx"
file_name = datetime.today().strftime("%Y%m%d") + "_監理站領牌數據全.csv"
full_path = export_path + file_name
# print(full_path)

In [ ]:
final["出廠年月"] = pd.to_datetime(final["出廠年月"], yearfirst=True)

In [ ]:
final["發照日"] = pd.to_datetime(final["發照日"], yearfirst=True)

In [ ]:
final = final[~(final["出廠年月"].isna()) & ~(final["發照日"].isna())]

In [ ]:
final["保留與否"] = final.apply(lambda x: "Drop" if (int(x["發照日"].year) - int(x["出廠年月"].year)) > 2 else "Keep", axis=1)

In [ ]:
final = final[final["保留與否"] == "Keep"]

In [ ]:
final = final[['品牌1', '品牌2', '來源', '車型', 'Tire size', '發照日', '出廠年月', '性別', '車重',
       '車主出生年', '車輛類型1', '車輛類型2', '使用類型', '城市', '區', '區域', '監理站', '車輛類型3']]

In [ ]:
final.to_csv(full_path, index=False)

In [ ]:
final.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\監理站新車領牌數據\20220822_監理站領牌數據202009-202209.xlsx", index=False)

## **尺寸表整理**

In [ ]:
new_tire_size = result[["出廠年月", "品牌2", "車型", "輪胎尺寸", "Tire size"]]

In [ ]:
new_tire_size = new_tire_size.drop_duplicates(subset=["出廠年月", "品牌2", "車型", "輪胎尺寸"])

In [ ]:
new_tire_size["出廠年月"] = new_tire_size["出廠年月"].dt.year

In [ ]:
new_tire_size.to_excel(r"C:\Users\kc.hsu\Python workspace\sales_related\New Car Registration Data\cross_tables\20220822_TIRE_SIZE_INDEX_NEW.xlsx", index=False)

In [ ]:
new_tire_size

In [ ]:
new_tire_size["Extract tire size 2"] = new_tire_size["輪胎尺寸"].str.extract(r"((\d{2,3}|\d{1,2}\D00|\d{4}|\d{2,3}\D|\d\D|\d\D\d)(|/\d{2}|/\d{2} |/\d{2}Z)(R\d{2}|RF\d{2}|r\d{2}| ZR\d{2})(\.\d{1}| |C|))")[0]

In [ ]:
new_tire_size["Extract tire size 2"] = new_tire_size["Extract tire size 2"].astype(str).map(lambda x: x.replace("Z", "").replace("C", "").replace("F", "").replace("r", "R").replace(" ", "").replace("f", ""))

In [ ]:
new_tire_size.to_excel(r"C:\Users\kc.hsu\Python workspace\sales_related\New Car Registration Data\cross_tables\20220822_TIRE_SIZE_INDEX_NEW.xlsx", index=False)

In [ ]:
result[result["輪胎尺寸"] == "215/"]

In [ ]:
result[result["車型名稱"] == "TOYOTA HIACE"]

### 新規格整理

In [ ]:
new_df = pd.read_clipboard()

In [ ]:
new_df["出廠年"] = new_df["出廠年月"].map(lambda x: x[:4])

In [ ]:
new_df = new_df[new_df["Tire size"].isna()]

In [ ]:
new_df.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\監理站新車領牌數據\20220915_需要整理的規格.xlsx", index=False)

## 整理大卡客車品牌

In [ ]:
df = pd.read_csv(r"D:\kc.hsu\OneDrive - Bridgestone\數據\監理站新車領牌數據\20220915_監理站領牌數據全.csv")

In [ ]:
truck = df[df["車輛類型3"] == "Truck/Bus"]

In [ ]:
truck_keys = ['MITSUBISHI', '順益', 'TOYOTA', 'ISUZU', '達富', 'SCANIA', '昇鋒猛獅',
       'M-BENZ', '成運', '賽德卡', 'MAN', 'MITSUBISHI FUSO', 'MITSUBISHI ROSA',
       '依維柯嘉馬', 'VOLVO', 'IVECO', 'HYUNDAI', '台灣華菱', '大金龍', 'SITRAK',
       '唐榮申沃', '創奕', '台灣海格', '華德', 'FUSO', '弘鉅', '總盈金龍', '健益', '馨盛亞星',
       '凱勝比亞迪', 'IVECO MAGIRUS', '金旅', '威達利', 'DONGFENG', '鴻華先進', '百盛']
truck_values = ['FUSO', 'FUSO', 'HINO', 'ISUZU', 'DAF', 'SCANIA', '昇鋒猛獅',
       'M-BENZ', '成運', '賽德卡', 'MAN', 'FUSO', 'FUSO',
       'IVECO', 'VOLVO', 'IVECO', 'HYUNDAI', '台灣華菱', '大金龍', '賽德卡',
       '唐榮申沃', '創奕', '台灣海格', '華德', 'FUSO', '弘鉅', '總盈金龍', '健益', '馨盛亞星',
       '凱勝比亞迪', 'IVECO', '金旅', '威達利', 'DONGFENG', '鴻華先進', '百盛']
truck_dict = dict(zip(truck_keys, truck_values))

In [ ]:
truck["品牌2"] = truck["品牌2"].map(truck_dict)

In [ ]:
wo_truck = df[~(df["車輛類型3"] == "Truck/Bus")]

In [ ]:
result = pd.concat([wo_truck, truck])

In [ ]:
result.to_csv(r"D:\kc.hsu\OneDrive - Bridgestone\數據\監理站新車領牌數據\20220915_監理站領牌數據全.csv", index=False)

## Fuzzywuzzy模糊匹配測試

In [6]:
reference = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\Yahoo汽車OE size.xlsx")

In [87]:
result

,品牌,發照日,戶籍郵遞區號,車種,車型名稱,出廠年月,性別,車重,輪胎尺寸,車主出生年,原始監理單位代碼,引擎與車身號碼加密,比對車型
0,LAMBORGHINI,2022-10-03,242,112,URUS,2022-07-01,女,na,285/45ZR21/315/40ZR2,1972,80,7C2C8BF7DF05D22E478200A739BCE8A7017E3E1E74F9CC...,"(Land Cruiser Prado 2.7越野玩家版, 68)"
1,VOLVO,2022-10-03,950,112,XC40 T2,2022-08-01,女,1.625,235/55R18,1967,81,87BCCC18D833EA5814BC202C2175A9F8990A60FD4E48C3...,"(GR86 2.4 6MT, 41)"
2,MAZDA,2022-10-03,221,112,CX-5 2WD-P,2022-08-01,男,1.565,225/55R19,1987,70,B071DCB538C362541379CD780011D55EFED1F292013F2E...,"(Camry 2.5 Hybrid尊爵, 86)"
3,中華,2022-10-03,251,113,TS-FK135-01,2022-09-01,法人,6.86,265/70R19.5,000,70,5DC865D577E0E20C77B54A775488B3B2CCB92020236EAB...,"(Vios 1.5雅緻, 46)"
4,BMW,2022-10-03,251,112,X6 XDRIVE40I,2022-05-01,法人,2.092,275/45 R20 305/40 R2,000,52,DC03BD5934EF55599BF22FC214A8BCB561B87E947CD427...,"(Granvia 6人座旗艦, 38)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36621,VOLVO,2022-10-31,114,112,S60 B4,2022-07-01,法人,1.675,235/45R18,000,20,BF06A9B6C241B9210D2F078F9201994E4AE98A4D34FD04...,"(GR86 2.4 6MT, 45)"
36622,VOLVO,2022-10-31,114,112,S60 B4,2022-08-01,法人,1.675,235/45R18,000,20,5D4A60BF700CCFFDDD3F89EA7E2A0B812ABAFBE258ABF3...,"(GR86 2.4 6MT, 45)"
36623,VOLVO,2022-10-31,114,112,S60 B4,2022-08-01,法人,1.675,235/45R18,000,20,C17BDACB1BE8658A6226A95CEE14C50327D95E384E2AE5...,"(GR86 2.4 6MT, 45)"
36624,VOLVO,2022-10-31,115,172,XC60 B4,2022-07-01,法人,1.855,235/60R18,000,20,83783684B29102B4B027CD7A27ED60CE260EA3895E8464...,"(GR86 2.4 6MT, 39)"
